### Information Retrieval

This is the document retrieval and sentence retrieval part of the project.

####  1. Unpack the zip file.
    
Unpack the 'wiki-pages-text.zip' in the current directory.

In [5]:
import zipfile
def unpack():
    with zipfile.ZipFile('wiki-pages-text.zip') as file:
        file.extractall()
unpack()

#### 2. Load file. 

Load the training dataset and the wiki txt file.

In [1]:
import os
import json
with open('train.json', 'r') as f:  # load training dataset
        train_data = json.load(f)   
print("Length of the train data is: " + str(len(train_data)))

with open('devset.json', 'r') as f1:  # load dev dataset
        dev_data = json.load(f1) 
print("Length of the dev data is: " + str(len(dev_data)))

with open('devset_result.json', 'r') as f2:  # store result 
        res_data = json.load(f2) 
print("Length of the dev result data is: " + str(len(res_data)))

with open('test-unlabelled.json', 'r') as f3:  # store result 
     test_data = json.load(f3) 
print("Length of the test data is: " + str(len(test_data)))
        
# appeand all the wiki txt sentences to one document
def loadfile(folder): 
    document = []
    list_of_files = os.listdir(folder)
    
    for file in list_of_files:
        try:
            filename = os.path.join(folder, file)
            with open(filename, 'r') as doc:
                for line in doc:
                    document.append(line)     
        except Exception as e:
            print("No files found here!")
            raise e
    return document
document = loadfile("wiki-pages-text")

print("Length of the document is: " + str(len(document)))

Length of the train data is: 145449
Length of the dev data is: 5001
Length of the dev result data is: 5001
Length of the test data is: 14997
Length of the document is: 25248397


#### 3. Preprocess 
Preprocess includes: strip punctuations, tokenize,lemma, lower case, remove stop words.

In [2]:
import nltk
import re
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

stop_words = set(stopwords.words('english')) 

def preprocess(sentence):
    norm_sentence = []
    sentence = re.sub(r'[^\w\s]', '', sentence) # strip punctuations: remove ',' '.',etc.
    tokens = nltk.tokenize.word_tokenize(sentence)  
                
    for token in tokens:
        token = lemmatize(token)
        token = token.lower() 
        if (token == "no" or token == "not"):  # keep no from stop words as it is useful for analysis
            norm_sentence.append(token)  
        if token not in stop_words:         # remove stop words
            norm_sentence.append(token)                  
    return norm_sentence

#### 4. Build inverted index.
Compute document term frequency, build <b> inverted index </b> and then uses BM25 to rank.



In [3]:
from collections import Counter

def doc_term_freq(doc):
    doc_term_freqs = []
    for sentence in doc:
        doc_term_freqs.append(Counter(sentence)) 
    return doc_term_freqs

class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.total_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.total_doc_len += doc_len
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

#### 5. BM25 
Okapi BM25 function is used to rank the sentences.

In [12]:
from math import log, sqrt

def BM25(query, index, vocab, k=5):
    k1 = 1.2
    k3 = 1.5
    b = 0.75
    # scores stores doc ids and their scores
    scores = Counter()
    length_average = index.total_doc_len/index.total_num_docs  # average length of document 
    
    for term in query:
        if term not in list(vocab.keys()):     # skip if the query word is not in the vocab
            continue
        N = index.num_docs()      # N: total number of documents
        ft = index.f_t(term)       # ft: document frequency of term
        docs = index.docids(term)    # docs: all doc ids that contain the term 
        dft = index.freqs(term)      # dft: all document freqs of the term 
        fqt = Counter(query)[term]   # fqt: frequency of tern t in query
        
        for num, docid in enumerate(docs):                                                                       
            fdt = dft[num]                                #fdt: frequency of term t in document d   
            length = sqrt(abs(index.doc_len[docid]))      # length: length of the doc
            #BM25 consists of three parts
            idf = log((N - ft + 0.5)/(ft + 0.5))
            tf = ((k1 + 1) * fdt)/(k1 * ((1-b) + b * length/length_average) + fdt)
            query_tf = (k3 + 1) * fqt / (k3 + fqt)
            
            scores[docid] +=  idf * tf * query_tf           
    return scores.most_common(k)

#### TF-idf function.

In [13]:
# tf idf  function in homework 1
# given a query and an index returns a list of the k highest scoring documents as tuples containing <docid,score>
def tfidf(query, index, vocab, k=5):
    
    # scores stores doc ids and their scores
    scores = Counter()
    for term in query:   
        
        if term not in list(vocab.keys()):   # skip if the query word is not in the vocab
            continue
            
        N = index.num_docs()      # N: total number of documents
        ft = index.f_t(term)       # ft: document frequency of term
        docs = index.docids(term)    # docs: all doc ids that contain the term 
        dft = index.freqs(term)      # dft: all document freqs of the term         
        
        for num, docid in enumerate(docs):                # num: index used for iterate the docs                                                        
            fdt = dft[num]                                #fdt: frequency of term t in document d   
            length = sqrt(abs(index.doc_len[docid]))      # length: length of the doc
            tfidf = log(1 + fdt)*log(N/ft)                 # tfidf: construct the score formula 
            scores[docid] += tfidf/length                  # score: the final score

    return scores.most_common(k)

#### 6. Use page identifier to reduce the search space.

Build a dictionary that consists the <b>page identifier</b> as the key and the document index(ranges form 0 to 25248397 ） as value.

Examples looks like: "Alexander McNair" : [1,2,3,4...18] 

<b>To do:</b> the keywords list may also consider synonyms for each keyword.


In [6]:
final_dic = {}

for id, doc in enumerate(document):
    final_dic.setdefault((doc.split()[0].replace("_", " ")), []).append(id)

keys = list(final_dic.keys())
print("Length of keys = {}".format(len(keys)))

Length of keys = 5396106


#### 7. Retrieval Evidence
<b>7.1</b> Given a query, tokenize it first, then for each token in the query, try to find it in the keys list. 


To do: smater select the in the range of its alphabet.

In [7]:
# Given a query, returns a list that contains all the document index values.
def extract_sentences(query, keys, final_dic):
    retrievaled_sentences = []
    for word in  nltk.tokenize.word_tokenize(query):
        for key in keys:
            if word in key:
                retrievaled_sentences.append(final_dic[key])
            
    retrievaled_sentences = [item for sublist in retrievaled_sentences for item in sublist]
    return retrievaled_sentences


<b>7.2</b> If there is a match or substring match, then retrieve that sentence by using index value to find the raw sentence in txt file. 

<b>7.3</b> Find all the related sentences by this way and then builds a inverted index and 
uses BM25 to rank and to retrieval top K sentences as the evidence.


In [14]:
# Use the result from above step, find all the raw sentences in txt file.
# return a list of the doc id orderd by rnaking tfidf or bm25 score.

def retrieval_evidence(query, keys, final_dic):
    processed_doc = [] # processed_docs stores the list of processed docs
    vocab = {}
    unique_id = 0
    rank_result = []
    
    retrievaled_sentences = extract_sentences(query,keys,final_dic)
    
    # find the row sentences and save them in processed_doc
    for retrievaled_sentence in retrievaled_sentences:
        norm_sentence = preprocess(document[retrievaled_sentence])
        for token in norm_sentence:
            if token not in vocab:
                vocab.update({token: unique_id})     
                unique_id = unique_id + 1
        processed_doc.append(norm_sentence) 
    
    # calculate doc term freqs and build an inverted index
    doc_term_freqs = doc_term_freq(processed_doc)
    invindex = InvertedIndex(vocab, doc_term_freqs)
    
    processed_query = preprocess(query)
    bm25_results = BM25(processed_query, invindex, vocab)
    tfidf_results = tfidf(processed_query, invindex, vocab)
    
    for rank, res in enumerate(tfidf_results):
        # print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],document[res[0]]))
        rank_result.append((res[0]))
    return rank_result

# test_query = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."
# test_query = "Alexander Alatskivi"
# print(retrieval_evidence(test_query,keys[:100],final_dic))

#### 8 Write the result to json file.

The dataset used is 'devset.json', the predicted result is 'devset_result.json'.


In [18]:
def get_evidence(res_data):
    for key in list(res_data)[:10]:
        res_data[key]["evidence"] = []
        tfidf_result = retrieval_evidence(res_data[key]["claim"],keys[:5000],final_dic)
        for res in tfidf_result:
            res_data[key]["evidence"].append([document[res].split()[0], document[res].split()[1]])
    return res_data

#testing, for top 10 instances in the dev, and only consider top 100 sentences in the documents.
predicted_train = get_evidence(res_data)

#for key in list(predicted_train)[:10]:
 #   print(predicted_train[key])
  #  print("\n")